# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_data_df = pd.read_csv("../output_data/city_data_df.csv")
city_data_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,mar del plata,-38.0023,-57.5575,48.09,77,65,11.01,AR,1655194151
1,1,vaini,-21.2000,-175.2000,71.76,94,20,3.44,TO,1655194377
2,2,praia da vitoria,38.7333,-27.0667,60.76,86,100,3.29,PT,1655194378
3,3,atuona,-9.8000,-139.0333,76.62,79,57,19.55,PF,1655194379
4,4,hithadhoo,-0.6000,73.0833,83.75,69,62,6.55,MV,1655194381
...,...,...,...,...,...,...,...,...,...,...
568,568,kibala,9.1104,18.3463,83.03,61,95,6.44,TD,1655195118
569,569,hvide sande,56.0045,8.1294,60.10,82,75,11.50,DK,1655195119
570,570,mana,5.6592,-53.7767,75.00,90,100,3.49,GF,1655195120
571,571,fairbanks,64.8378,-147.7164,63.25,53,0,1.01,US,1655194851


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# create lists for heatmap
humidity = city_data_df["Humidity"]
city_loc = city_data_df[["Lat","Lng"]]

# create heatmap layer on gmap fig
fig = gmaps.figure(center = [0,0] ,zoom_level = 2)
heat_layer = gmaps.heatmap_layer(city_loc, weights=humidity,
                                dissipating=False, max_intensity=100, 
                                point_radius = 3)
# add layer
fig.add_layer(heat_layer)

# check it exists
print(fig)

Figure(layout=FigureLayout(height='420px'))


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# create df with desired weather conditions and drop null values
clean_city_df  = city_data_df.loc[(city_data_df["Max Temp"]>70)
                            &(city_data_df["Max Temp"]<80)
                            &(city_data_df["Wind Speed"]<10)
                            &city_data_df["Cloudiness"]==0,:].dropna()

clean_city_df


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,mar del plata,-38.0023,-57.5575,48.09,77,65,11.01,AR,1655194151
1,1,vaini,-21.2000,-175.2000,71.76,94,20,3.44,TO,1655194377
2,2,praia da vitoria,38.7333,-27.0667,60.76,86,100,3.29,PT,1655194378
3,3,atuona,-9.8000,-139.0333,76.62,79,57,19.55,PF,1655194379
4,4,hithadhoo,-0.6000,73.0833,83.75,69,62,6.55,MV,1655194381
...,...,...,...,...,...,...,...,...,...,...
568,568,kibala,9.1104,18.3463,83.03,61,95,6.44,TD,1655195118
569,569,hvide sande,56.0045,8.1294,60.10,82,75,11.50,DK,1655195119
570,570,mana,5.6592,-53.7767,75.00,90,100,3.49,GF,1655195120
571,571,fairbanks,64.8378,-147.7164,63.25,53,0,1.01,US,1655194851


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df= clean_city_df[["City", "Country", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,mar del plata,AR,-38.0023,-57.5575,
1,vaini,TO,-21.2000,-175.2000,
2,praia da vitoria,PT,38.7333,-27.0667,
3,atuona,PF,-9.8000,-139.0333,
4,hithadhoo,MV,-0.6000,73.0833,
...,...,...,...,...,...
568,kibala,TD,9.1104,18.3463,
569,hvide sande,DK,56.0045,8.1294,
570,mana,GF,5.6592,-53.7767,
571,fairbanks,US,64.8378,-147.7164,


In [6]:
# create base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# set parameters for url search
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key}

# Iterate through cities
for index, row in hotel_df.iterrows():
    # get lat, lng from dataframe
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # Make request
    print(f"Retrieving results for city: {row['City']}")
    hotel_name = requests.get(base_url, params=params).json()
    
    # hotel_name = hotel_name['results']
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
    
    print("------------")

# Display hotel names on dataframe
hotel_df



Retrieving results for city: mar del plata
------------
Retrieving results for city: vaini
------------
Retrieving results for city: praia da vitoria
------------
Retrieving results for city: atuona
------------
Retrieving results for city: hithadhoo
------------
Retrieving results for city: punta arenas
------------
Retrieving results for city: grand forks
------------
Retrieving results for city: qalyub
------------
Retrieving results for city: kapaa
------------
Retrieving results for city: maumere
------------
Retrieving results for city: khatanga
------------
Retrieving results for city: vardo
------------
Retrieving results for city: port elizabeth
------------
Retrieving results for city: mazagao
------------
Retrieving results for city: santa maria
------------
Retrieving results for city: pangnirtung
------------
Retrieving results for city: tasiilaq
------------
Retrieving results for city: arys
------------
Retrieving results for city: torbay
------------
Retrieving results 

,City,Country,Lat,Lng,Hotel Name
0,mar del plata,AR,-38.0023,-57.5575,Gran Hotel Mar del Plata
1,vaini,TO,-21.2000,-175.2000,Keleti Beach Resort
2,praia da vitoria,PT,38.7333,-27.0667,Hotel Branco I
3,atuona,PF,-9.8000,-139.0333,Villa Enata
4,hithadhoo,MV,-0.6000,73.0833,Scoop Guest House
...,...,...,...,...,...
568,kibala,TD,9.1104,18.3463,hotel safari
569,hvide sande,DK,56.0045,8.1294,Hvide Sande Hotel
570,mana,GF,5.6592,-53.7767,LE SAMANA
571,fairbanks,US,64.8378,-147.7164,Westmark Fairbanks Hotel & Conference Center


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [8]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
! jupyter nbextension enable --py --sys-prefix gmaps
! jupyter nbextension enable --py gmaps
! jupyter nbextension enable --py widgetsnbextension

Config option `kernel_spec_manager_class` not recognized by `EnableNBExtensionApp`.
Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok
Config option `kernel_spec_manager_class` not recognized by `EnableNBExtensionApp`.
Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok
Config option `kernel_spec_manager_class` not recognized by `EnableNBExtensionApp`.
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok
